# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [2]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_imdb_reviews(movie_urls):
    all_reviews_with_ratings = []
    for movie_url, release_date, stars in movie_urls:
        reviews_with_ratings = []
        page = 1
        while len(reviews_with_ratings) < 1000:
            response = requests.get(movie_url + f'&start={page * 10}')
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                review_containers = soup.find_all('div', class_='review-container')
                if not review_containers:
                    break
                for container in review_containers:
                    review_text = container.find('div', class_='text show-more__control').text.strip()
                    rating_element = container.find('span', class_='rating-other-user-rating')
                    if rating_element:
                        rating = rating_element.find('span').text.strip()
                    else:
                        rating = 'Not Rated'
                    reviews_with_ratings.append({'Review': review_text, 'Rating': rating})
                    if len(reviews_with_ratings) >= 1000:
                        break
                print(f"Collected {len(reviews_with_ratings)} reviews for {movie_url}")
                page += 1
            else:
                print(f"Failed to retrieve data from {movie_url}")
                break
        all_reviews_with_ratings.extend(reviews_with_ratings)
    return all_reviews_with_ratings

def save_reviews_to_csv(reviews, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Review'])
        for review in reviews:
            writer.writerow([review['Review']])


def main():
    movie_data = [
        ('https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2', '2023-01-01', 8.5),
        # Add URLs, release dates, and stars for more movies if needed
    ]
    reviews_with_ratings = scrape_imdb_reviews(movie_data)
    if len(reviews_with_ratings) >= 1000:
        save_reviews_to_csv(reviews_with_ratings[:1000], 'movie_reviews_with_ratings.csv')
        print("Successfully saved reviews to movie_reviews_with_ratings.csv")
    else:
        print("Failed to collect enough reviews.")
if __name__ == '__main__':
    main()

Collected 25 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 50 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 75 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 100 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 125 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 150 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 175 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 200 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 225 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 250 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 275 reviews for https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ql_2
Collected 300 reviews for https://www.imdb.com/title/tt1517268/revie

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [9]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load the data from CSV
df = pd.read_csv('/content/movie_reviews_with_ratings.csv')
print("Original Data:")
print(df.head())

# Step 1: Remove noise, special characters, and punctuations
df['clean_review'] = df['Review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

print("Step 1: Remove noise, special characters, and punctuations:")
print(df[['Review', 'clean_review']].head())

# Step 2: Remove numbers
df['clean_review'] = df['clean_review'].apply(lambda x: re.sub(r'\d+', '', x))
print("Step 2: Remove numbers:")
print(df[['Review', 'clean_review']].head())

# Step 3: Remove stopwords
stop_words = set(stopwords.words('english'))
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(word for word in x.split() if word.lower() not in stop_words))
print("Step 3: Remove stopwords:")
print(df[['Review', 'clean_review']].head())

# Step 4: Lowercase all texts
df['clean_review'] = df['clean_review'].apply(lambda x: x.lower())
print("Step 4: Lowercase all texts:")
print(df[['Review', 'clean_review']].head())

# Step 5: Stemming
porter = PorterStemmer()
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(porter.stem(word) for word in x.split()))
print("Step 5: Stemming:")
print(df[['Review', 'clean_review']].head())

# Step 6: Lemmatization
lemmatizer = WordNetLemmatizer()
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))
print("Step 6: Lemmatization:")
print(df[['Review', 'clean_review']].head())

df.to_csv('/content/movie_reviews_with_ratings.csv', index=False)
print("\nCleaned Data:")
print(df[['Review', 'clean_review']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original Data:
                                              Review
0  Margot does the best with what she's given, bu...
1  Before making Barbie (2023), Greta Gerwig sing...
2  The quality, the humor, and the writing of the...
3  As much as it pains me to give a movie called ...
4  As a woman that grew up with Barbie, I was ver...
Step 1: Remove noise, special characters, and punctuations:
                                              Review  \
0  Margot does the best with what she's given, bu...   
1  Before making Barbie (2023), Greta Gerwig sing...   
2  The quality, the humor, and the writing of the...   
3  As much as it pains me to give a movie called ...   
4  As a woman that grew up with Barbie, I was ver...   

                                        clean_review  
0  Margot does the best with what shes given but ...  
1  Before making Barbie  Greta Gerwig singlehande...  
2  The quality the humor and the writing of the m...  
3  As much as it pains me to give a movie called .

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [40]:
import spacy
from spacy import displacy
from nltk import pos_tag, word_tokenize, RegexpParser
import nltk
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Text to analyze
text = df['clean_review'].iloc[0]

doc = nlp(text)

# Part-of-speech tagging and counting
pos_counts = {'NOUN': 0, 'VERB': 0, 'ADJ': 0, 'ADV': 0}
for token in doc:
    if token.pos_ in pos_counts:
        pos_counts[token.pos_] += 1

# Print POS counts
print("POS Counts:")
for pos, count in pos_counts.items():
    print(f"{pos}: {count}")

# Dependency Parsing
print("\nDependency Parsing Trees:")
for sent in doc.sents:
    for token in sent:
        print(token.text, "-->", token.dep_, "-->", token.head.text)

# Named Entity Recognition (NER) and counting
ner_counts = {}
for ent in doc.ents:
    ner_counts[ent.label_] = ner_counts.get(ent.label_, 0) + 1

# Print NER counts
print("\nNamed Entity Recognition:")
for ent_label, count in ner_counts.items():
    print(f"{ent_label}: {count}")


    """
Constituency parsing breaks a sentence into phrases and clauses based on grammar rules, showing their hierarchical structure in a tree. For example, in "Barbi directs two films," "Barbi" is the subject, "directs" is the verb, and "two films" is the object.

Dependency parsing focuses on word relationships, representing them as directed links. In the same sentence, "directs" is the root word, "Barbi" is the subject, and "two films" is the object, shown through arrows indicating dependencies.

In essence, constituency parsing reveals the sentence's grammar-based hierarchy, while dependency parsing emphasizes word connections. Both provide valuable insights into sentence structure and relationships."""

POS Counts:
NOUN: 27
VERB: 13
ADJ: 9
ADV: 4

Dependency Parsing Trees:
margot --> advmod --> best
best --> advmod --> given
she --> nsubj --> given
given --> ROOT --> given
film --> compound --> satir
disappoint --> compound --> market
market --> compound --> fun
fun --> compound --> satir
quirki --> compound --> satir
satir --> compound --> movi
homag --> compound --> movi
movi --> nsubj --> start
start --> dobj --> given
way --> advmod --> start
end --> conj --> given
overdramat --> compound --> speech
speech --> nsubj --> end
end --> nsubj --> make
clearli --> compound --> tri
tri --> nsubj --> make
make --> ccomp --> end
audienc --> nsubj --> feel
feel --> ccomp --> make
someth --> advmod --> left
left --> amod --> everyon
everyon --> nsubj --> feel
feel --> ccomp --> feel
confus --> nsubj --> say
say --> ccomp --> feel
i --> nsubj --> m
m --> ccomp --> say
crotcheti --> npadvmod --> old
old --> amod --> man
man --> dobj --> given
i --> nsubj --> m
m --> ROOT --> m
woman --> attr -

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [43]:
'''I tried working on another topic in the first question, but I was not able to gather more data from other sources. However, IMDB review gathering is a little easier compared to others, and clearing the text data is the easy part, but I face some issues in the 3rd question on Constituency Parsing and Dependency Parsing.'''

'I tried working on another topic in the first question, but I was not able to gather more data from other sources. However, IMDB review gathering is a little easier compared to others, and clearing the text data is the easy part, but I face some issues in the 3rd question on Constituency Parsing and Dependency Parsing.'